# A1

In [1]:
import nltk
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords 
from collections import Counter
import random
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE
import re

In [2]:
#accessing  drive and opening file + decoding in read mode.

myfile1 = open("Project Gutenberg's The Complete Works of Jane Austen, by Jane Austen.txt","r")

#Replace newline with space and convert to lower case
text=myfile1.read().replace("\n"," ")
text=text.lower()

In [3]:
#tokenizing sentences
stopwords_en = set(stopwords.words("english"))
sents = nltk.sent_tokenize(text)
sents_rm_stopwords = []
for sent in sents:
  sents_rm_stopwords.append(' '.join(w for w in nltk.word_tokenize(sent) if w.lower() not in stopwords_en))

In [4]:
#removing useless symbols
process = []
tdata=[]
for sent in sents_rm_stopwords:
  out = re.sub(r'[^\w\d\s]+','', sent)
  out=str(out)
  process.append('<s> '+out+'</s>')
  tdata.append(out)

In [5]:
#test-train split of data 
data_train, data_test = train_test_split(process, test_size=0.2, random_state=32)


In [6]:
nest_pro=[]
for sent in data_train:
    temp = re.split(r'[" "]+', sent)
    for i in temp:
        if i=="":
            temp.remove(i)
    nest_pro.append(temp)

In [7]:
#word tokenizing of each sentence in train data 
nest_train=[]
for sent in data_train:
  temp = re.split(r'[" "]+', sent)
  for i in temp:
    if i=="":
      temp.remove(i)
  nest_train.append(temp)
for i in nest_train:
  if i==[""]:
    nest_train.remove(i)

In [8]:
#word tokenizing of each sentence in test data 
nest_test=[]
for sent in data_test:
  temp = re.split(r'[" "]+', sent)
  for i in temp:
    if i=="":
      temp.remove(i)
  nest_test.append(temp)
for i in nest_test:
  if i==[""]:
    nest_test.remove(i)


In [9]:
def getwordngrams(processed_tokenized_tweet, k):
	word_n_grams = []
  
	for i in range(k, k + 1):
		word_i_grams = [" ".join(processed_tokenized_tweet[j:j+i]) for j in range(len(processed_tokenized_tweet) - (i-1))]
		word_n_grams= word_i_grams + word_n_grams
	return word_n_grams
unigram =[]
tempp=[]
for d in nest_pro:
    tempp.append(getwordngrams(d,1))
for l in tempp:
    unigram+=l
unidict=Counter(unigram)
tokens=0
for i in unidict.keys():
    tokens+=unidict[i]

In [10]:
bigram =[]
tempp=[]
for d in nest_pro:
  tempp.append(getwordngrams(d,2))
for l in tempp:
  bigram+=l
bidict=Counter()
for i in range(len(unigram)-1):
    b=(unigram[i],unigram[i+1])
    bidict[b]=bidict.get(b,0)+1

In [11]:
trigram =[]
tempp=[]
for d in nest_pro:
  tempp.append(getwordngrams(d,3))
for l in tempp:
  trigram+=l
tridict=Counter()
for i in range(len(unigram)-2):
    b=(unigram[i],unigram[i+1],unigram[i+2])
    tridict[b]=tridict.get(b,0)+1

In [12]:
quadgram =[]
tempp=[]
for d in nest_pro:
  tempp.append(getwordngrams(d,4))
for l in tempp:
  quadgram+=l
quaddict=Counter()
for i in range(len(unigram)-3):
    b=(unigram[i],unigram[i+1],unigram[i+2],unigram[i+3])
    quaddict[b]=quaddict.get(b,0)+1

In [13]:
umle={}
for i in unidict.keys():
    umle[i]=unidict[i]/tokens
#example
print("Example: unigram mle for alice:",umle["alice"])

#bigrams
bmle={}
for (i,j) in bidict.keys():
    bmle[(i,j)]=bidict[(i,j)]/unidict[i]
#print(big_mle) 
#example
print("Example: unigram mle for 'give away':",bmle['give', 'away'])

#trigrams
tmle={}
for (i,j,k) in tridict.keys():
    tmle[(i,j,k)]=tridict[(i,j,k)]/bidict[(i,j)]
#quadgrams
qmle={}
for (i,j,k,l) in quaddict.keys():
    qmle[(i,j,k,l)]=quaddict[(i,j,k,l)]/tridict[(i,j,k)]

Example: unigram mle for alice: 2.903979904459061e-06
Example: unigram mle for 'give away': 0.011965811965811967


In [14]:
#How many n-grams are possible and how many actually exist?
to=len(unidict)

print("          {0: <10} {1: <20}".format("Actual", "Possible"))
print("          {0: <10} {1: <20}".format("------", "--------"))
print("unigram   {0: <10} {1: <20}".format(to,to))
print("bigram    {0: <10} {1: <20}".format(len(bidict), to**2))
print("trigram   {0: <10} {1: <20}".format(len(tridict), to**3))
print("quadgram  {0: <10} {1: <20}".format(len(quaddict), to**4))

          Actual     Possible            
          ------     --------            
unigram   14877      14877               
bigram    201191     221325129           
trigram   281915     3292653944133       
quadgram  330675     48984812726866641   


In [15]:
def findnextword(prob,next_word): 
    s=0
    for i in prob:
        s+=i
    for i in prob:
        i=i/s
    a=np.random.multinomial(1,prob,size=1).tolist()
    return next_word[a[0].index(1)]
def bigprediction(cw):
    prob=[]
    next_word=[]
    for (b1,b2) in bmle.keys():
        if (b1==cw):
            prob.append(bmle[(b1,b2)])
            next_word.append(b2)
    return prob,next_word

def trigprediction(cw,cw1):
    prob=[]
    next_word=[]
    for (t1,t2,t3) in tmle.keys():
        if ((t1,t2)==(cw,cw1)):
            prob.append(tmle[(t1,t2,t3)])
            next_word.append(t3)
    return prob,next_word

def quadprediction(cw,cw1,cw2):
    prob=[]
    next_word=[]
    for (q1,q2,q3,q4) in qmle.keys():
        if ((q1,q2,q3)==(cw,cw1,cw2)):
            prob.append(qmle[(q1,q2,q3,q4)])
            next_word.append(q4)
    return prob,next_word

In [16]:
def sent_gen(n): #n=2 for bigram, 3 for trigram..
    sent_length=random.randint(8,20) #length of sentence
    sentence=["<s>"]
    length=0
    cw,cw1,cw2="<s>","",""  #current words 0,1,2 for predicting next word
    
    if(n==2):
        while(length<sent_length):
            prob,next_word=bigprediction(cw)
            f=findnextword(prob,next_word)
            
            sentence.append(f)
            cw=f
            length+=1
            if(cw=="</s>"):
                break
    
    if(n==3):
        prob,next_word=bigprediction(cw)  #predict first word using bigram
        f=findnextword(prob,next_word)
        sentence.append(f)
        cw1=f
        length+=1
        
        while(length<sent_length): #predict all other words using trigram
            prob,next_word=trigprediction(cw,cw1)
            f=findnextword(prob,next_word)
            sentence.append(f)
            cw=cw1
            cw1=f
            length+=1
            if(cw1=="</s>"):
                break
    if(n==4):
        prob,next_word=bigprediction(cw) #predict first word using bigram
        f=findnextword(prob,next_word)
        sentence.append(f)
        cw1=f
        
        prob,next_word=trigprediction(cw,cw1) #predict second word using trigram
        f=findnextword(prob,next_word)
        sentence.append(f)
        cw2=f
        length+=2
        
        while(length<sent_length): #predict all other words using quadgram
            prob,next_word=quadprediction(cw,cw1,cw2)
            f=findnextword(prob,next_word)
            sentence.append(f)
            cw=cw1
            cw1=cw2
            cw2=f
            length+=1
            if(cw2=="</s>"):
                break
    s=" ".join(sentence)
    return s
print("Bigram sentences:")        
for i in range(5):
    a=sent_gen(2)
    print(str(i+1)+'.'+a[4:len(a)-4])
print("\n Trigram sentences:")
for i in range(5):
    a=sent_gen(3) 
    print(str(i+1)+'.'+a[4:len(a)-4])

print("\n Quadgram sentences:")  
for i in range(5):
    a=sent_gen(4)
    print(str(i+1)+'.'+a[4:len(a)-4])

Bigram sentences:
1.appear prominently whenever spoke seemed resolved instead going farther 
2.oh 
3.whatever may like emma quite unawares quite unpardonable folly conduct 
4.allowed promote father lived miles 
5.catherine s spirits would different ways love suit be

 Trigram sentences:
1.arrival seemed afford real satisfaction good heart settling family females cottage satisfaction sportsman sportsman though esteems sex sportsmen likewise o
2.fanny led willingly though impossible fanny s head therefore oppose mother s sake forbear recalling remembrance unhappy situation repaired impris
3.sisters intended walk said mary 
4.let us quarrel past 
5.henrietta exactly state recentlyimproved views freshformed happiness 

 Quadgram sentences:
1.elizabeth excessively disappointed set heart seeing lakes still thought might time enough 
2.also handsome replied elizabeth young man ought 
3.heartiness warmth sincerity anne delighted sad want blessings home 
4.every body dearest always hand probabl

#### The statements are grammatically senseless. the sentences get smaller with increase in 'n' inn ngram

In [17]:
nest_test=[]
for sent in data_test:
    temp = re.split(r'[" "]+', sent)
    for i in temp:
        if i=="":
            temp.remove(i)
    nest_test.append(temp)
test_words =[]
tempp=[]
for d in nest_test:
    tempp.append(getwordngrams(d,1))
for l in tempp:
    test_words+=l

In [18]:
def perplexity(n,sentence):
    prob=0
    for i in range(len(sentence)-n):
        try:
            if(n==1):
                prob+=np.log2(umle[sentence[i]])
            if(n==2):
                prob+=np.log2(bmle[(sentence[i],sentence[i+1])])
            if(n==3):
                prob+=np.log2(tmle[(sentence[i],sentence[i+1],sentence[i+2])])
            if(n==4):
                prob+=np.log2(qmle[(sentence[i],sentence[i+1],sentence[i+2],sentence[i+3])])
        except:
            continue
            
    l =  prob/len(sentence)
    return np.power(2, -l)

In [19]:
def test_mean_perplexity(n, test_dataset):
    test_per=0
    for sentence in test_dataset:
        test_per+=perplexity(n,sentence)
    return test_per/len(test_dataset)

def test_perplexity(n, test_dataset):
    test_per=0
    for sentence in test_dataset:
        test_per+=perplexity(n,sentence)
    return test_per

In [20]:
token_sent_test = nest_test
print("           {0: <20} {1: <20}".format("Perplexity", "Mean Perplexity"))
print("           {0: <20} {1: <10}".format("----------", "---------------"))
print(" unigram   {0: <20} {1: <10}".format(test_perplexity(1, token_sent_test),test_mean_perplexity(1, token_sent_test)))
print(" bigrams   {0: <20} {1: <10}".format(test_perplexity(2, token_sent_test),test_mean_perplexity(2, token_sent_test)))
print(" trigrams  {0: <20} {1: <10}".format(test_perplexity(3, token_sent_test),test_mean_perplexity(3, token_sent_test)))
print(" quadgrams {0: <20} {1: <10}".format(test_perplexity(4, token_sent_test),test_mean_perplexity(4, token_sent_test)))

           Perplexity           Mean Perplexity     
           ----------           ---------------
 unigram   5407911.921802663    771.2367258703171
 bigrams   59922.37882859499    8.545690078236593
 trigrams  8541.955119832657    1.2181909754467566
 quadgrams 7066.518793681491    1.0077750704052326


# A3

#### The Neural Network will mostly have a better output than the classical appoarch. This is because it can tackle sentences that it has never seen. It also uses context of all of the previous words in the sentences, hence it would be better then classical approach 